In [1]:
#### cria ambiente spark e delta tables 
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("AppSaulo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.debug.maxToStringFields", 100000)

spark = configure_spark_with_delta_pip(builder).getOrCreate()



:: loading settings :: url = jar:file:/home/saulos_pires/.local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/saulos_pires/.ivy2/cache
The jars for the packages stored in: /home/saulos_pires/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bb985cec-a9af-494c-96aa-fde78a1133f8;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 241ms :: artifacts dl 12ms
	:: modules in use:
	io.delta#delta-core_2.12;1.2.1 from central in [default]
	io.delta#delta-storage;1.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| searc

In [26]:
#df = spark.read.csv('/tmp/map.csv')
#df.createOrReplaceTempView('planilha')
#df = spark.sql('select * from planilha')
#df.show()

#df2 = spark.read.format("text").load("/tmp/mapped.txt")
#df2.createOrReplaceTempView('mapeado')
df2 = spark.sql('select a.*, b.* from planilha as a left join mapeado as b on a._c2 = b.value where b.value is null')
df2.show()

#df3 = spark.sql('select count(*) from (select distinct _c2 from planilha) as a ')
#df3.show()

#df4 = spark.sql('select count(*) from (select distinct value from mapeado) as a ')
#df4.show()

#df5 = spark.sql('select a.*, b.* from planilha as a right join mapeado as b on a._c2 = b.value where a._c2 is null')
#df5.show()




+------+--------+----------+--------------------+-----+
|   _c0|     _c1|       _c2|                 _c3|value|
+------+--------+----------+--------------------+-----+
|Banner|Multiple|  Multiple|COURSE_MASTER_COU...| null|
|Banner|Multiple|  Multiple|COURSE_MASTER_COU...| null|
|Banner| SBRPERS|BN_SBRPERS|STUDENT_PIDM_PERS...| null|
|Banner| SGRADVR|BN_SGRADVR| STUDENT_ADVISOR_LNK| null|
|Banner| SGRADVR|BN_SGRADVR| STUDENT_ADVISOR_SAT| null|
|Banner| SGRCHRT|BN_SGRCHRT|STUDENT_PIDM_COHO...| null|
|Banner| SGRCHRT|BN_SGRCHRT|STUDENT_PIDM_COHO...| null|
+------+--------+----------+--------------------+-----+



In [17]:
df_parquet = spark.sql('''create database if not exists db_parquet ''')
df_parquet = spark.sql('''create table if not exists db_parquet.InvoiceParquet using parquet location "/home/saulos_pires/parquet/ds_zz_Invoice_folder"''')
df_parquet = spark.sql('''select distinct InvoiceNumber from db_parquet.InvoiceParquet  where InvoiceNumber <> 0536884625''')
df_parquet.show()

#0536884625

+-------------+
|InvoiceNumber|
+-------------+
|   0536913516|
|   0536885915|
|   0536889340|
|   0536858651|
|   0536920352|
|   0536820814|
|   0603818755|
|   0536829121|
|   0536856582|
|   0536919708|
|   0536918130|
|   0536839307|
|   0536910428|
|   0536819051|
|   0536894469|
|   0536878221|
|   0536842104|
|   0536883930|
|   0536911917|
|   0536827362|
+-------------+
only showing top 20 rows



In [21]:
df_parquet.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("/home/saulos_pires/dt_tables/InvoiceDelta") 




In [3]:
df_delta = spark.sql('''create database if not exists db_delta ''')
df_delta = spark.sql('''create table if not exists db_delta.InvoiceDelta using delta location "/home/saulos_pires/dt_tables/InvoiceDelta"''')
df_delta = spark.sql('''select * from db_delta.InvoiceDelta where InvoiceNumber = 0536884625''')
df_delta.show()

+-------------+
|InvoiceNumber|
+-------------+
|   0536884625|
+-------------+



In [6]:
df_insert = spark.sql('''select distinct InvoiceNumber from db_parquet.InvoiceParquet  where InvoiceNumber = 0536884625''')
df_insert.show()

+-------------+
|InvoiceNumber|
+-------------+
|   0536884625|
+-------------+



In [7]:
deltaTablePeople = DeltaTable.forPath(spark, '/home/saulos_pires/dt_tables/InvoiceDelta')
#deltaTablePeopleUpdates = DeltaTable.forPath(spark, '/tmp/delta/people-10m-updates')

dfUpdates = df_insert

deltaTablePeople.alias('people') \
  .merge(
    dfUpdates.alias('updates'),
    'people.InvoiceNumber = updates.InvoiceNumber'
  ) \
  .whenMatchedUpdate(set =
    {
      "InvoiceNumber": "updates.InvoiceNumber"
    }
  ) \
  .whenNotMatchedInsert(values =
    {
      "InvoiceNumber": "updates.InvoiceNumber"
    }
  ) \
  .execute()

In [34]:
df = spark.read.parquet('/home/saulos_pires/parquet/ds_zz_Invoice_folder/ds_zz_Invoice')
df.createOrReplaceTempView('produto')

In [48]:
###df_resultado = spark.sql('select product_code, row_number() over(partition by product_code order by ins_gmt_ts desc ) as linha from produto ')
df_teste = spark.sql("select * from (select SKU, row_number() over(partition by SKU order by InvoiceDate desc ) as linha from produto ) where linha = 1 order by 1 desc")

In [49]:
df_teste.show()

+-------+-----+
|    SKU|linha|
+-------+-----+
|7006263|    1|
|7006262|    1|
|7006242|    1|
|7001862|    1|
|7001695|    1|
|7001692|    1|
|7001685|    1|
|7000281|    1|
|7000278|    1|
|6018052|    1|
|6017353|    1|
|6017352|    1|
|6017351|    1|
|6017350|    1|
|6017349|    1|
|6017348|    1|
|6017345|    1|
|6017344|    1|
|6017343|    1|
|6017342|    1|
+-------+-----+
only showing top 20 rows



In [5]:
df.show()

+------------------+--------------------+-------------+-------------+----------------------+-----------------+---------------+----------+------------+------------+-----------------+------------+--------------------------+---------------------+---------------+------------------+-------------+--------------+----------------+------+----------+----------+--------------+----------+--------------+-----------------+-----------------+-------+
|      product_code|        product_name|category_code|category_name|brand_family_name_code|brand_family_name|brand_name_code|brand_name|package_code|package_name|package_type_code|package_type|package_material_type_code|package_material_type|pack_volumetric|price_segment_code|price_segment|product_volume|marketplace_flag|status|src_sys_id|ins_usr_id|    ins_gmt_ts|upd_usr_id|    upd_gmt_ts|      load_job_nr|src_sys_region_nm|act_flg|
+------------------+--------------------+-------------+-------------+----------------------+-----------------+------------